In [7]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv('Iris.csv')
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
class Node():
    
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):

    # node constructor
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

Tree class

In [ ]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):

        #constructor

        # initialize root of the tree
        self.root = None

        # stopping conditions

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

        def build_tree(self, dataset, curr_depth=0):
            # build tree recursive function

            X, Y = dataset[:,:-1], dataset[:,-1]
            num_samples, num_features = np.shape(X)

            # split tree until stopping conditions are met

            if num_samples >= self.min_samples_split and curr_depth <= max_depth:
                # find the best split
                best_split = self.get_best_split(dataset, num_samples, num_features)
                if best_split['info_gain']>0:
                    # recur left
                    left_subtree = self.build_tree(best_split['dataset_left'], curr_depth+=1)
                    # recur 
                    right_subtree = self.build_tree(best_split['dataset_left'], curr_depth+=1)
                    # return decision node
                    return Node(best_split['feature_index'], best_split['threshold'], left_subtree, right_subtree, best_split['info_gain'])
                
            leaf_value = self.calculate_leaf_value(Y)
            return Node(value=leaf_value)
        
        def get_best_split(self, dataset, num_samples, num_features):

            best_split = {}
            max_info_gain = -float('inf')

            for feature_index in range(num_features):
                feature_values = dataset[:, feature_index]
                possible_thresholds = np.unique(feature_values)

                for threshold in possible_thresholds:
                    # get current split
                    dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                    if len(dataset_left)>0 and len(dataset_right) >0:
                        y, left_y, right_y = dataset[:,-1], dataset_left[:, -1], dataset_right[:, -1]
                        # compute information gain using gini index
                        curr_info_gain = self.information_gain(y, left_y, right_y, 'gini')
                        # update best_split if needed
                        if curr_info_gain>max_info_gain:
                            best_split['feature_index'] = feature_index
                            best_split['threshold'] = threshold
                            best_split['dataset_left'] = dataset_left
                            best_split['dataset_right'] = dataset_right
                            best_split['info_gain'] = curr_info_gain
                            max_info_gain = curr_info_gain

                return best_split
            
            def split(self, dataset, feature_index, threshold):
                # split data
                dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
                dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
                return dataset_left, dataset_right
            
            def information_gain(self, parent, l_child, r_child, mode='entropy'):

                weight_l = len(l_child) / len(parent)
                weight_r = len(r_child) /len(parent)
                if mode == 'gini':
                    gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
                else:
                    gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
                return gain
            
            def entropy(self, y):

                class_labels = np.unique(y)
                entropy = 0
                for cls in class_labels:
                    p_cls = len(y[y == cls]) / len(y)
                    entropy += -p_cls * np.log2(p_cls)
                return entropy
            
            def gini_index(self, y):
                # compute gini index
                class_labels = np.unique(y)
                gini = 0